In [1]:
from pyspark.sql import SparkSession

# (8 cores, 16gb per machine) x 5 = 40 cores

# New API
spark_session = SparkSession\
        .builder\
        .master("local") \
        .appName("Question_A4")\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

In [2]:
textEN = spark_context.textFile("/home/ubuntu/LDSA/lab2/europarl-v7.sv-en.en")
textSW = spark_context.textFile("/home/ubuntu/LDSA/lab2/europarl-v7.sv-en.sv")
pipedEN = textEN.pipe("/home/ubuntu/LDSA/lab2/tools/tokenizer.perl -l en")
pipedSW = textSW.pipe("/home/ubuntu/LDSA/lab2/tools/tokenizer.perl -l sv")

In [3]:
indexedEN = pipedEN.zipWithIndex()

In [4]:
#indexedEN.take(5)

In [5]:
indexedENInverted = indexedEN.map(lambda x: (x[1], x[0]))

In [6]:
#indexedENInverted.take(5)

In [7]:
indexedSWInverted = pipedSW.zipWithIndex()\
.map(lambda x: (x[1], x[0]))

In [8]:
#indexedSWInverted.take(5)

In [9]:
joinedCorpae = indexedSWInverted.join(indexedENInverted)

In [10]:
joinedCorpae.take(5)

[(0, ('Återupptagande av sessionen', 'Resumption of the session')),
 (1835010,
  ('Jag hoppas att han tar sig litet tid för att gå in på det .',
   'I hope that he will take a little time to address this issue before we close .')),
 (1310724,
  ('De måste få möjlighet att göra ett upplyst val med utgångspunkt från tydlig information .',
   'They must be able to make an informed choice on the basis of clear information .')),
 (786438,
  ('Det är önskvärt att SIS II installeras och tas i drift så snart som möjligt , och jag anser att kommissionen bör meddela parlamentet om det uppstår eventuella förseningar och skälen till dem .',
   'I believe that it would be desirable for SIS II to be up and running as soon as possible and I feel that the Commission should notify Parliament of any delays that occur and the reasons behind those delays .')),
 (262152,
  ('Fördelningen av personbilar har alltid behandlats särskilt och sektoriellt och om jag tar det snabbt , skulle detta system kunna ha v

In [80]:
wordsSplit = joinedCorpae.map(lambda x: (x[0], (x[1][0].split(" "), x[1][1].split(" "))))

In [81]:
wordsSplit.take(5)

[(0,
  (['Återupptagande', 'av', 'sessionen'],
   ['Resumption', 'of', 'the', 'session'])),
 (1835010,
  (['Jag',
    'hoppas',
    'att',
    'han',
    'tar',
    'sig',
    'litet',
    'tid',
    'för',
    'att',
    'gå',
    'in',
    'på',
    'det',
    '.'],
   ['I',
    'hope',
    'that',
    'he',
    'will',
    'take',
    'a',
    'little',
    'time',
    'to',
    'address',
    'this',
    'issue',
    'before',
    'we',
    'close',
    '.'])),
 (1310724,
  (['De',
    'måste',
    'få',
    'möjlighet',
    'att',
    'göra',
    'ett',
    'upplyst',
    'val',
    'med',
    'utgångspunkt',
    'från',
    'tydlig',
    'information',
    '.'],
   ['They',
    'must',
    'be',
    'able',
    'to',
    'make',
    'an',
    'informed',
    'choice',
    'on',
    'the',
    'basis',
    'of',
    'clear',
    'information',
    '.'])),
 (786438,
  (['Det',
    'är',
    'önskvärt',
    'att',
    'SIS',
    'II',
    'installeras',
    'och',
    'tas',
    'i'

In [60]:
wordsSplit.count()

1862234

In [82]:
wordsSplitFiltered = wordsSplit.filter(lambda element: not " " == element[1][1] or not " " == element[1][0])\
.filter(lambda element: len(element[1][1]) < 6)\
.filter(lambda element: len(element[1][0]) < 6)\
.filter(lambda element: len(element[1][0])==len(element[1][1]))

In [83]:
wordsSplitFiltered.take(5)

[(1311156, (['Vi', 'måste', 'agera', '.'], ['We', 'must', 'act', '.'])),
 (1835460,
  (['Den', 'andra', 'är', 'invandringen', '.'],
   ['The', 'second', 'is', 'immigration', '.'])),
 (468,
  (['Jag', 'förklarar', 'debatten', 'avslutad', '.'],
   ['The', 'debate', 'is', 'closed', '.'])),
 (524826, (['(', '1', ')'], ['(', '1', ')'])),
 (1049526,
  (['Inkomna', 'dokument', ':', 'se', 'protokollet'],
   ['Documents', 'received', ':', 'see', 'Minutes']))]

In [109]:
rdd1 = wordsSplitFiltered.flatMap(lambda x: x[1][0])
rdd2 = wordsSplitFiltered.flatMap(lambda x: x[1][1])
wordsJoined = rdd1.zip(rdd2)

In [110]:
wordsJoined.count()

88076

In [116]:
wordsJoined.take(5)

[('Vi', 'We'), ('måste', 'must'), ('agera', 'act'), ('.', '.'), ('Den', 'The')]

In [132]:
wordsJoined.map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)\
.sortByKey(True).take(10)

[(('', '.'), 416),
 (('!', '!'), 714),
 (('!', "'"), 2),
 (('!', ','), 5),
 (('!', '.'), 211),
 (('!', ':'), 1),
 (('!', '?'), 4),
 (('!', 'I.'), 1),
 (('!', 'Söderman'), 1),
 (('!', 'you'), 1)]

In [134]:
wordsJoined.map(lambda word: (word, 1)) \
            .reduceByKey(lambda a, b: a + b)\
            .sortBy(lambda x: x[1], False).take(30)

[(('.', '.'), 15439),
 (('(', '('), 3532),
 ((')', ')'), 3416),
 (('Applåder', 'Applause'), 3021),
 (('avslutad', 'closed'), 2881),
 (('är', 'is'), 2588),
 (('.', ''), 2330),
 (('?', '?'), 1711),
 (('Jag', 'The'), 1319),
 (('debatten', 'is'), 1319),
 (('förklarar', 'debate'), 1313),
 (('härmed', 'is'), 1213),
 (('är', 'debate'), 1186),
 (('Debatten', 'The'), 1185),
 ((':', ':'), 933),
 (('Det', 'That'), 789),
 (('!', '!'), 714),
 (('Jag', 'I'), 581),
 (('1', '1'), 537),
 (('Vi', 'We'), 530),
 (('Varför', 'Why'), 474),
 (('Detta', 'This'), 473),
 (('2', '2'), 468),
 (('Det', 'It'), 448),
 (('inte', 'not'), 441),
 (('3', '3'), 420),
 (('', '.'), 416),
 (('–', 'The'), 355),
 (('Debatten', 'debate'), 354),
 (('är', 'are'), 340)]